In [72]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset, random_split

import pandas as pd
import numpy as np

In [73]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Ti'

In [74]:
class SimpleConvNet(nn.Module):
    def __init__(self, in_channels):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 128)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [75]:
N = 8
C = 32
batch_size = 1

input_tensor = torch.rand(batch_size, C, N, N)

In [76]:
input_tensor.shape

torch.Size([1, 32, 8, 8])

In [77]:
model = SimpleConvNet(in_channels=C)

# Pass the input tensor through the model
output = model(input_tensor)

print("Output shape:", output.shape)
print("Output value:", output.item())

Output shape: torch.Size([1, 1])
Output value: 0.48863983154296875


In [78]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

In [79]:
class RandomTensorDataset(Dataset):
    def __init__(self, num_samples, input_size):
        self.num_samples = num_samples
        self.input_size = input_size

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        input_tensor = torch.rand(*self.input_size)
        target = torch.rand(1)
        
        return input_tensor, target


In [87]:
num_samples = 1000
batch_size = 16
input_size = (C, N, N)

random_dataset = RandomTensorDataset(num_samples=num_samples, input_size=input_size)

train_size = int(0.6 * len(random_dataset))
val_size = int(0.2 * len(random_dataset))
test_size = len(random_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(random_dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [88]:
train_dataset.__getitem__(1)[0].shape

torch.Size([32, 8, 8])

In [89]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Training Loss: {epoch_loss}')

model.eval() 
val_loss = 0.0
with torch.no_grad():
    for inputs, targets in val_loader:
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        val_loss += loss.item() * inputs.size(0)
val_loss /= len(val_loader.dataset)
print(f'Validation Loss: {val_loss}')

test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        test_loss += loss.item() * inputs.size(0)
test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss}')

Training Loss: 0.6938337063789368
Training Loss: 0.6933249648412069
Training Loss: 0.6922160108884176
Training Loss: 0.6942118684450785
Training Loss: 0.6933810782432556
Training Loss: 0.693081243832906
Training Loss: 0.6929723421732584
Training Loss: 0.6935933971405029
Training Loss: 0.6933888387680054
Training Loss: 0.6930167182286581
Validation Loss: 0.692316358089447
Test Loss: 0.6927933835983277


In [ ]:
torch.save(model.state_dict(), 'test_model.pth')